In [66]:
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node ad6b0acc-2bf0-4a27-bb7a-64d9d8bda21d

In [67]:
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import numpy as np
from tqdm import tqdm

In [68]:
from threading import Timer

class RepeatedTimer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False

In [76]:
def motors(left, right):
    return {
        "motor.left.target": [left],
        "motor.right.target": [right],
    }

def get_data():
    thymio_data.append({"sensor":node["prox.horizontal"],
                        "left_speed":node["motor.left.speed"],
                        "right_speed":node["motor.right.speed"]})

async def get_speed():
    if acquire_data:
        await node.wait_for_variables() # wait for Thymio variables values
        rt = RepeatedTimer(Ts, get_data) # it auto-starts, no need of rt.start()

        try:
            await client.sleep(Ts)
        finally:
            rt.stop() # better in a try/finally block to make sure the program ends!
            node.send_set_variables(motors(0, 0))

async def rotate(theta, motor_speed): #theta is in radians
    direction_rot=(theta>=0)-(theta<0)
    await node.set_variables(motors(motor_speed*direction_rot, -motor_speed*direction_rot))
    # wait time to get theta 1.44 is the factor to correct
    time=(theta)*100/motor_speed*1.44
    await(client.sleep(time))
    # stop the robot
    await node.set_variables(motors(0, 0))

def pos_estimation(thymio_data, dt, pos0_x, pos0_y, theta0):
    #current variables
    left_speed = [x["left_speed"] for x in thymio_data]
    right_speed = [x["right_speed"] for x in thymio_data]

    if len(left_speed) == 0:
        return pos0_x, pos0_y, theta0

    #speed estimation
    speed = (right_speed[-1] + left_speed[-1])/2
    angular_speed = (right_speed[-1] - left_speed[-1])/2
    theta = theta0 + angular_speed*dt

    #position estimation
    pos_x = pos0_x + speed*np.cos(theta)*dt
    pos_y = pos0_y + speed*np.sin(theta)*dt

    return pos_x, pos_y, theta

In [81]:
Ts = 1
thymio_data = []
threshold=100
acquire_data=True
pos_x, pos_y, theta = 0, 0, 0
thymio_data.clear()

while True:
    #await client.sleep(5)
    node.send_set_variables(motors(50, 50))
    await get_speed()
    pos_x, pos_y, theta = pos_estimation(thymio_data, Ts, pos_x, pos_y, theta)
    print(pos_x, pos_y)
    prox_sensor = [x["sensor"][2] for x in thymio_data]   
    if (len(prox_sensor)!=0) and (prox_sensor[-1]>threshold):
        acquire_data=False
        break

12.481136160381958 42.19266808517809
54.96269787955098 32.834948603358114
82.24796432589204 75.32923333615689
51.84636324030845 115.65283252364316
4.880597025002004 125.56327165665739


In [73]:
await tdmclient.notebook.stop()

ValueError: Function <function _pre_run_cell at 0x7f8be4798d30> is not registered as a pre_run_cell callback